# JWST Rogue Path Tool Example with Program 

## Imports

In [ ]:
import numpy as np
import pathlib

from jwst_rogue_path_tool.program_data_parser import aptJsonFile
from jwst_rogue_path_tool.detect_claws import aptProgram
from jwst_rogue_path_tool.constants import PROJECT_DIRNAME

HOMEDIR = pathlib.Path.home()

## Working with APT JSON Files
Below we will show how to inspect APT JSON files using AptJSONFile

In [ ]:
aptJsonFile?

### Opening APT JSON File and Displaying Table Names

In [ ]:
apt_json_filename = pathlib.Path(PROJECT_DIRNAME) / "data" / "APT_test_4RPtool.records.json"

# Parse the APT JSON file to obtain useful information for Claw Tool
parsed_json = aptJsonFile(apt_json_filename)

parsed_json.tablenames

### Selecting Table and Displaying Contents

In [ ]:
tablename = "exposures"
parsed_json.data[tablename]  # This is a list of dictionaries

### Creating DataFrame of Table Contents

In section 2.1 we displayed the contents of a table as a list of dictionaries. Here we make use of the `build_dataframe` to create a DataFrame to make the contents more legible and gives us access to all of the useful `pandas` methods and attributes

In [ ]:
exposures_df = parsed_json.build_dataframe(tablename)
exposures_df  # this is a dataframe

## Creating a Program object with the JWST Rogue Path Tool with AptProgram Class

The examples below showcase the difference use cases for the JWST Rogue Path Tool.

In [ ]:
aptProgram?

### Cycle through all angles of attitude for a single observation of a program.

The example below is taking the our APT JSON and stepping through each angle of attitude in one degree steps.

In [ ]:
program = aptProgram(apt_json_filename, angular_step=1.0)

In [ ]:
program.run()

In [ ]:
program.observations.data  # Show ALL observation data

In [ ]:
program.observations.supported_observations  # Only show the supported observations for JWST Rogue Path Tool
program.observations.supported_observations.keys()  # Observation numbers to select data on.

In [ ]:
# Select a single observation of interest
observation = program.observations.data[1]
observation['exposure_frames'].data
# show pandas df for single exposure frame

#### Show Valid Angle Ranges per Expsoure

In [ ]:
supported_observations = program.observations.supported_observations.keys()
for obs_number in supported_observations:
    program.plot_exposures(program.observations.data[obs_number])

#### Show Valid Angles Ranges at Observation Level

This will show valid angle ranges across all exposures in a single observation. If angles ranges are valid in one exposure but not in others, it will not be shown at this level.

In [ ]:
supported_observations = program.observations.supported_observations.keys()
for obs_number in supported_observations:
    program.plot_observation(program.observations.data[obs_number])

### Plotting a Single Angle of Attitude in V2 & V3 Space

The figure below shows what the tool is searching for when sweeping angles. Here for a single angle we show the V2 & V3 space of the targets in our source catalog as well as the susceptibility region. The intensity of targets that fall in this region are dependent on the location where of the target in the susceptibility region and the brightness of the target.

In [ ]:
from jwst_rogue_path_tool.plotting import plot_fixed_angle_regions

angles = np.arange(100, 106)

for angle in angles:
    plot_fixed_angle_regions(observation, angle)

### Plotting Flux as a Function of Attitude.

Below we show the calculated flux as a function of attitude angle. This figure also ties in the [JWST Backgrounds Tool](https://github.com/spacetelescope/jwst_backgrounds) to calculate the background of the target JWST is observering described in the APT file. By default, we set the parameters for the backgrounds tool to calculate 10% of the minimum background value. You can set the function and percentage via a dictionary and pass it to the AptProgram class.
~~~
jbt_params = [{"threshold":0.1, function:np.min}, {"threshold":0.2, "function":np.mean}, ..... {"threshold":value, "function":stats_function}]

AptProgram(apt_JSON_filename, angular_step=1.0, bkg_params=jbt_params)
~~~

In [ ]:
from jwst_rogue_path_tool.plotting import create_v3pa_vs_flux_plot

for obs_number in supported_observations:
    create_v3pa_vs_flux_plot(program.observations.data[obs_number])